In [9]:
import pandas as pd
import os
import shutil
import numpy as np
import glob
import subprocess

from QC.utils import shell_do

In [10]:
report = pd.read_csv('/data/CARD/PD/ibx_data/F4SEttehadieh_P1_Genotyping_Report.txt', sep='\s+')
report

/usr/local/Anaconda/envs/py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Index,Name,Address,Chr,Position,GenTrain,Score,Frac,A,Frac.1,...,FS04821406_FO0000173_APP_A673T_F06_17_C10.Theta,FS04821406_FO0000173_APP_A673T_F06_17_C10.R,FS04821407_FO0000174_APP_A673T_H08_17_C11.GType,FS04821407_FO0000174_APP_A673T_H08_17_C11.Score,FS04821407_FO0000174_APP_A673T_H08_17_C11.Theta,FS04821407_FO0000174_APP_A673T_H08_17_C11.R,PositiveControl_1_NA_NA.GType,PositiveControl_1_NA_NA.Score,PositiveControl_1_NA_NA.Theta,PositiveControl_1_NA_NA.R
0,1,1:100011159-T-G,95711256,1,100011159,0.824643,0.358333,0.141667,0.216667,0.283333,...,0.989883,BB,0.789367,0.941056,0.893555,NaN,NaN,NaN,NaN,NaN
1,2,1:10002775-GA,4697595,1,10002775,0.831192,0.333333,0.233333,0.200000,0.233333,...,1.335987,BB,0.401563,0.931935,1.124636,NaN,NaN,NaN,NaN,NaN
2,3,1:100122796-C-T,44771929,1,100122796,0.817369,0.341667,0.091667,0.325000,0.241667,...,1.658113,AA,0.777982,0.010333,2.147722,NaN,NaN,NaN,NaN,NaN
3,4,1:100152282-CT,91663346,1,100152282,0.791415,0.208333,0.225000,0.158333,0.408333,...,1.538377,BB,0.376958,0.947486,1.537969,NaN,NaN,NaN,NaN,NaN
4,5,1:100154376-GA,49790543,1,100154376,0.889701,0.350000,0.158333,0.191667,0.300000,...,0.738183,BB,0.493571,0.968484,0.611655,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1779814,1779815,Y:9384122-AC,39666258,Y,9384122,0.872065,0.125000,0.333333,0.266667,0.275000,...,0.687159,AA,0.471352,0.069092,0.527377,NaN,NaN,NaN,NaN,NaN
1779815,1779816,Y:9408770-G-T,89616381,Y,9408770,0.570935,0.433333,0.225000,0.158333,0.183333,...,1.956097,BB,0.191398,0.663617,2.482646,NaN,NaN,NaN,NaN,NaN
1779816,1779817,Y:9785736-A-G,30774949,Y,9785736,0.887508,0.416667,0.091667,0.216667,0.275000,...,0.495367,AA,0.463757,0.067339,0.391138,NaN,NaN,NaN,NaN,NaN
1779817,1779818,Y:9847423-T-C,58633139,Y,9847423,0.709727,0.233333,0.208333,0.275000,0.283333,...,0.540618,BB,0.295714,0.861137,0.586958,NaN,NaN,NaN,NaN,NaN


In [5]:
basedir = f'/data/CARD/PD/ibx_data'
idat_dir = f'{basedir}/Raw_Data'
ped_dir = f'{basedir}/ped'
bed_dir = f'{basedir}/bed'
gtc_dir = f'{basedir}/gtc'
ref_dir = '../ref'
fasta = f'{ref_dir}/human_g1k_v37.fasta'
swarm_scripts_dir = f'{basedir}/swarm_scripts'

os.makedirs(ped_dir, exist_ok=True)
os.makedirs(bed_dir, exist_ok=True)
os.makedirs(gtc_dir, exist_ok=True)
os.makedirs(swarm_scripts_dir, exist_ok=True)

ilmn_files_path = '/data/CARD/PD/GP2/ilmn_files'
bpm = f'{ilmn_files_path}/NeuroBooster_20042459_A1.bpm'
egt = f'{ilmn_files_path}/NBSCluster_file_n1393_011921.egt'
iaap = f'{ilmn_files_path}/iaap-cli/iaap-cli'

In [6]:
with open(f'{swarm_scripts_dir}/idat_to_ped.swarm', 'w') as f:
        
    idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{ped_dir}/ \
-f {idat_dir}/ \
-p \
-t 16'
        
    f.write(f'{idat_to_ped_cmd}\n')
f.close()

In [7]:
!swarm -f {swarm_scripts_dir}/idat_to_ped.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

16704530


In [34]:
with open(f'{swarm_scripts_dir}/idat_to_gtc.swarm', 'w') as f:
        
    idat_to_gtc_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{gtc_dir}/ \
-f {idat_dir}/ \
-g \
-t 16'
        
    f.write(f'{idat_to_gtc_cmd}\n')
f.close()

In [35]:
!swarm -f {swarm_scripts_dir}/idat_to_gtc.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

17041626


In [29]:
gtc_list = [s.split('.')[0] for s in glob.glob(f'{gtc_dir}/*.gtc')]

with open(f'{swarm_scripts_dir}/gtc2vcf.swarm','w') as f:
    for gtc in gtc_list:
        gtc2vcf_cmd = f'/data/vitaled2/GenoTools/GTCtoVCF/gtc_to_vcf.py --gtc-paths {gtc}.gtc --manifest-file {bpm} --genome-fasta-file {fasta} --include-attributes {{GT,GQ,BAF,LRR}} --skip-indels --output-vcf-path {gtc}.vcf'
        f.write(f'{gtc2vcf_cmd}\n')
f.close()

In [31]:
!swarm -f {swarm_scripts_dir}/gtc2vcf.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

17205191


In [38]:
for gtc in gtc_list:
    vcf_view_cmd = f'bcftools view -Oz -o {gtc}.vcf.gz {gtc}.vcf'
    shell_do(vcf_view_cmd)

Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204835450051_R02C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204835450051_R02C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204842400028_R06C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204842400028_R06C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204842400119_R03C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204842400119_R03C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204842400014_R08C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204842400014_R08C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204820990004_R08C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204820990004_R08C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204842400137_R01C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204842400137_R01C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204835450051_R08C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204835450051_R08C01.vcf
Executing: bcftools view -Oz -o /data/CAR

In [42]:
for gtc in gtc_list:
    vcf_tabix_cmd = f'tabix -p vcf {gtc}.vcf.gz'
    shell_do(vcf_tabix_cmd)

Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204835450051_R02C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400028_R06C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400119_R03C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400014_R08C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204820990004_R08C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400137_R01C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204835450051_R08C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400103_R07C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204820990004_R03C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400003_R03C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400139_R05C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400028_R01C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400103_

In [8]:
# !bcftools merge --merge all {gtc_dir}/*.vcf.gz -Ov -o {gtc_dir}/ibx.vcf
bcftools_merge_cmd = f'bcftools merge --merge all {gtc_dir}/*.vcf.gz -Ov -o {gtc_dir}/ibx.vcf'
subprocess.run(bcftools_merge_cmd, stdout=subprocess.PIPE, shell=True)

CompletedProcess(args='bcftools merge --merge all /data/CARD/PD/ibx_data/gtc/*.vcf.gz -Ov -o /data/CARD/PD/ibx_data/gtc/ibx.vcf', returncode=0, stdout=b'')

In [71]:
line_count = 0
with open(f'{gtc_dir}/ibx.vcf', "r") as ifile:
    for line in ifile:
        line_count += 1
        if line.startswith("#CHROM"):
            break


vcf = pd.read_csv(f'{gtc_dir}/ibx.vcf', sep='\s+', skiprows=line_count-1)
vcf.rename(columns={'#CHROM':'CHROM'}, inplace=True)
IIDs = [x.split('/')[-1] for x in gtc_list]
vcf_melt = vcf.melt(id_vars=['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO'], value_vars=IIDs, value_name='metrics')
vcf_melt[['GT','GQ','BAF','LRR']] = vcf_melt.metrics.str.split(':', expand=True)
vcf_melt.drop(columns=['QUAL','FILTER','INFO'], inplace=True)
vcf_melt.to_csv(f'{gtc_dir}/ibx_variant_metrics.txt', sep='\t', index=False)

In [18]:
ped_filenames = [p.split('.')[0] for p in glob.glob(f'{basedir}/ped/*.ped')]
map_file = f'{ped_dir}/NeuroBooster_20042459_A1.map'
for filename in ped_filenames:
    ped = f'{filename}.ped'
    out_map = f'{filename}.map'
    if os.path.isfile(ped):
        shutil.copyfile(src=map_file, dst=out_map)
    else:
        print(f'{ped} does not exist!')
        print(f'{out_map} creation cancelled')

In [10]:
with open(f'{swarm_scripts_dir}/make_bed.swarm', 'w') as f:
    for filename in ped_filenames:
        ped = f'{filename}'
        make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {filename}'

        f.write(f'{make_bed_cmd}\n')
f.close()


In [12]:
!swarm -f {swarm_scripts_dir}/make_bed.swarm -g 32 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

16728607


In [13]:
with open(f"{bed_dir}/merge_bed.list", 'w') as f:
    for filename in ped_filenames:
        bed = f'{filename}'
        f.write(f'{bed}\n')
f.close()

with open(f"{swarm_scripts_dir}/merge.swarm", 'w') as f:

    plink_merge_cmd = f'\
plink \
--merge-list {bed_dir}/merge_bed.list \
--make-bed \
--out {bed_dir}/ibx'
    f.write(f"{plink_merge_cmd}")
f.close()

In [14]:
!swarm -f {swarm_scripts_dir}/merge.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

16741250


In [85]:
import pandas as pd
import os
import shutil
import zipfile

from QC.qc import callrate_prune, het_prune, sex_prune, related_prune, variant_prune, avg_miss_rates
from Ancestry.ancestry import run_ancestry, split_cohort_ancestry
from QC.utils import shell_do



In [86]:
geno_path = f'{bed_dir}/ibx'


In [6]:
steps = []

avg_miss = avg_miss_rates(geno_path, f'{geno_path}_missing')
avg_miss

callrate_out = f'{geno_path}_callrate'
callrate = callrate_prune(geno_path, callrate_out, mind=0.03)
steps.append(callrate)

sex_out = f'{callrate_out}_sex'
sex = sex_prune(callrate_out, sex_out)
steps.append(sex)


RUNNING: callrate_prune



Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx --mind 0.03 --make-bed --out /data/vitaled2/ibx_data/bed/ibx_callrate



RUNNING: sex_prune



Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --check-sex 0.25 0.75 --maf 0.05 --out /data/vitaled2/ibx_data/bed/ibx_callrate_sex_tmp1
Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --chr 23 --from-bp 2699520 --to-bp 154931043 --maf 0.05 --geno 0.05 --hwe 1E-5 --check-sex 0.25 0.75 --out /data/vitaled2/ibx_data/bed/ibx_callrate_sex_tmp2
Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --remove /data/vitaled2/ibx_data/bed/ibx_callrate_sex.fails --make-bed --out /data/vitaled2/ibx_data/bed/ibx_callrate_sex


In [4]:
1-0.008408835276531221

0.9915911647234688

In [5]:
1-0.008410770833333333


0.9915892291666667

In [9]:
# related
related_out = f'{callrate_out}_related'
related = related_prune(callrate_out, related_out)



RUNNING: related_prune



Executing: gcta --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --autosome --maf 0.05 --make-grm --out /data/vitaled2/ibx_data/bed/ibx_callrate_related_total_grm_tmp
Executing: gcta --grm /data/vitaled2/ibx_data/bed/ibx_callrate_related_total_grm_tmp --grm-cutoff 0.125 --make-grm --out /data/vitaled2/ibx_data/bed/ibx_callrate_related_unrelated_grm_tmp
Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --keep /data/vitaled2/ibx_data/bed/ibx_callrate_related_unrelated_grm_tmp.grm.id --make-bed --out /data/vitaled2/ibx_data/bed/ibx_callrate_related
Executing: gcta --grm /data/vitaled2/ibx_data/bed/ibx_callrate_related_total_grm_tmp --grm-cutoff 0.95 --make-grm --out /data/vitaled2/ibx_data/bed/ibx_callrate_related_duplicated_grm_tmp


{'pass': True,
 'step': 'related_prune',
 'metrics': {'related_count': 0, 'duplicated_count': 0},
 'output': {'pruned_samples': '/data/vitaled2/ibx_data/bed/ibx_callrate_related.related',
  'plink_out': '/data/vitaled2/ibx_data/bed/ibx_callrate_related'}}

In [103]:
idat_dir = f'{basedir}/ibx_plus_coriell'
ibx_only_samples = f'{basedir}/ped'


In [22]:
idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{idat_dir}/ \
-f {idat_dir}/ \
-p \
-t 16'

shell_do(idat_to_ped_cmd)

Executing: /data/CARD/PD/GP2/ilmn_files/iaap-cli/iaap-cli gencall /data/CARD/PD/GP2/ilmn_files/NeuroBooster_20042459_A1.bpm /data/CARD/PD/GP2/ilmn_files/NBSCluster_file_n1393_011921.egt /data/CARD/PD/ibx_data/ibx_plus_coriell/ -f /data/CARD/PD/ibx_data/ibx_plus_coriell/ -p -t 16


In [87]:
ped_filenames = [p.split('.')[0] for p in glob.glob(f'{idat_dir}/*.ped')]
# map_file = f'{ped_dir}/NeuroBooster_20042459_A1.map'
# for filename in ped_filenames:
#     ped = f'{filename}.ped'
#     out_map = f'{filename}.map'
#     if os.path.isfile(ped):
#         shutil.copyfile(src=map_file, dst=out_map)
#     else:
#         print(f'{ped} does not exist!')
#         print(f'{out_map} creation cancelled')

In [90]:
ibx_samples = [y.split('.')[0] for y in [x.split('/')[-1] for x in glob.glob(f'{ibx_only_samples}/*.fam')]]


In [24]:
for filename in ped_filenames:
    
    ped = f'{filename}'
    make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {filename}'
    
    shell_do(make_bed_cmd)

Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204825020102_R07C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204825020102_R07C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204835450116_R07C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204835450116_R07C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204842400004_R06C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204842400004_R06C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204701860056_R08C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204701860056_R08C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204835450020_R01C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204835450020_R01C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204701860109_R06C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204701860109_R06C01
Executing: plink --file /data/CARD

In [6]:
with open(f"{idat_dir}/merge_bed.list", 'w') as f:
    for filename in ped_filenames:
        bed = f'{filename}'
        f.write(f'{bed}\n')
f.close()

with open(f"{swarm_scripts_dir}/merge.swarm", 'w') as f:

    plink_merge_cmd = f'\
plink \
--merge-list {idat_dir}/merge_bed.list \
--make-bed \
--out {idat_dir}/ibx'
    f.write(f"{plink_merge_cmd}")
f.close()

In [8]:
!swarm -f {swarm_scripts_dir}/merge.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

16856865


In [104]:
geno_path = f'{idat_dir}/ibx'
steps=[]

callrate_out = f'{geno_path}_callrate'
callrate = callrate_prune(geno_path, callrate_out, mind=0.03)
steps.append(callrate)

sex_out = f'{callrate_out}_sex'
sex = sex_prune(callrate_out, sex_out)
steps.append(sex)



RUNNING: callrate_prune



Executing: plink --bfile /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx --mind 0.03 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate



RUNNING: sex_prune



Executing: plink --bfile /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate --check-sex 0.25 0.75 --maf 0.05 --out /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex_tmp1
Executing: plink --bfile /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate --chr 23 --from-bp 2699520 --to-bp 154931043 --maf 0.05 --geno 0.05 --hwe 1E-5 --check-sex 0.25 0.75 --out /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex_tmp2
Executing: plink --bfile /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate --remove /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex.fails --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex


In [108]:
# pruned_ibx_coriell = pd.read_csv('/data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex.fam',sep='\s+', names=['FID','IID','PAT','MAT','SEX','PHENO'])
# pruned_ibx_coriell[pruned_ibx_coriell.IID.isin(ibx_samples)]
callrate_outliers = pd.read_csv('/data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate.outliers',sep='\s+', header=None,names=['FID','IID'])
callrate_outliers[callrate_outliers.IID.isin(ibx_samples)]

,FID,IID
36,0,204842400139_R08C01


In [45]:
sexcheck = pd.read_csv('/data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex_tmp1.sexcheck',sep='\s+')
sexcheck

,FID,IID,PEDSEX,SNPSEX,STATUS,F
0,0,204697840001_R01C01,2,2,OK,-0.005508
1,0,204697840001_R02C01,1,1,OK,0.999700
2,0,204697840001_R03C01,2,2,OK,0.065470
3,0,204697840001_R04C01,2,2,OK,0.167800
4,0,204697840001_R05C01,1,1,OK,0.999600
...,...,...,...,...,...,...
2347,0,204854950032_R04C01,1,1,OK,0.999700
2348,0,204854950032_R05C01,1,1,OK,0.999600
2349,0,204854950032_R06C01,1,1,OK,0.999700
2350,0,204854950032_R07C01,2,2,OK,0.028070


In [46]:
sexcheck[sexcheck.IID.isin(ibx_samples)].STATUS.value_counts()

OK    95
Name: STATUS, dtype: int64

In [47]:
sexcheck.STATUS.value_counts()

OK         2335
PROBLEM      17
Name: STATUS, dtype: int64

In [39]:
coriell_metrics = pd.read_hdf('/data/CARD/PD/GP2/genotypes/coriell/clean/coriell.QC.metrics.h5', key='QC')

In [14]:
# now merge previously-called genotypes (ibx + coriell) to see how pruning works

coriell_geno = '/data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell'
ibx_geno = f'{bed_dir}/ibx'
ibx_coriell_geno = f'{ibx_geno}_coriell'

plink_merge_cmd = f'plink \
--bfile {ibx_geno} \
--bmerge {coriell_geno} \
--allow-no-sex \
--make-bed \
--out {ibx_coriell_geno}'

shell_do(plink_merge_cmd)


Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx --bmerge /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell --allow-no-sex --make-bed --out /data/CARD/PD/ibx_data/bed/ibx_coriell


In [16]:
geno_path = ibx_coriell_geno

In [17]:
steps = []

avg_miss = avg_miss_rates(geno_path, f'{geno_path}_missing')
avg_miss

callrate_out = f'{geno_path}_callrate'
callrate = callrate_prune(geno_path, callrate_out, mind=0.03)
steps.append(callrate)

sex_out = f'{callrate_out}_sex'
sex = sex_prune(callrate_out, sex_out)
steps.append(sex)

Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell --missing --out /data/CARD/PD/ibx_data/bed/ibx_coriell_missing


Average Missing Call Rate (lmiss): 0.01126679989325701
Average Missing Genotyping Rate (imiss): 0.011273141249999999

RUNNING: callrate_prune



Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell --mind 0.03 --make-bed --out /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate



RUNNING: sex_prune



Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate --check-sex 0.25 0.75 --maf 0.05 --out /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate_sex_tmp1
Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate --chr 23 --from-bp 2699520 --to-bp 154931043 --maf 0.05 --geno 0.05 --hwe 1E-5 --check-sex 0.25 0.75 --out /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate_sex_tmp2
Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate --remove /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate_sex.fails --make-bed --out /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate_sex
